In [1]:
import pandas as pd
import numpy as np
import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['tmdb_api_results_2010.json',
 'final_tmdb_data_2014.csv.gz',
 'tmdb_api_results_2011.json',
 'tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'tmdb_api_results_2001.json',
 'final_tmdb_data_2010.csv.gz',
 'title_basics.csv.gz',
 'tmdb_api_results_2014.json',
 'final_tmdb_data_2015.csv.gz',
 'tmdb_api_results_2015.json',
 'final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints',
 'final_tmdb_data_2013.csv.gz',
 'tmdb_api_results_2012.json',
 'title_akas.csv.gz',
 'Data',
 'titleR.csv.gz',
 'tmdb_api_results_2013.json',
 'final_tmdb_data_2011.csv.gz']

In [2]:
YEARS_TO_GET = [2016,2017,2018,2019]

In [3]:
basics= pd.read_csv('Data/title_basics.csv.gz')

In [4]:
errors = [ ]